In [88]:
import os
import sys
import ast
import logging
import yaml
import pkgutil 

try:
    # python 3
    AST_TRY = [ast.Try]
except AttributeError:
    # python 2.7
    AST_TRY = [ast.TryExcept, ast.TryFinally]

# this AST_QUESTIONABLE list comprises the various ways an import can be weird
# 1. inside a try/except block
# 2. inside a function
# 3. inside a class
AST_QUESTIONABLE = tuple(list(AST_TRY) + [ast.FunctionDef, ast.ClassDef])
AST_QUESTIONABLE = tuple(list(AST_TRY) + [ast.FunctionDef, ast.ClassDef])

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

pkg_data = yaml.load(
    pkgutil.get_data(__name__, 'pkg_data/pkg_data.yml').decode(),
    Loader=yaml.SafeLoader,
)

def _split(name):
    named_space = pkg_data['_NAMEDSPACE_MAPPING'].get(name)
    return named_space if named_space else name.split('.')[0]

# This is from https://github.com/ericdill/depfinder/blob/master/depfinder/main.py
class ImportFinder(ast.NodeVisitor):
    """Find all imports in an Abstract Syntax Tree (AST).
    Attributes
    ----------
    required_modules : list
        The list of imports that were found outside of try/except blocks,
        function definitions and class definitions
    sketchy_modules : list
        The list of imports that were found inside of try/except blocks,
        function definitions and class definitions
    imports : list
        The list of all ast.Import nodes in the AST
    import_froms : list
        The list of all ast.ImportFrom nodes in the AST
    """
    def __init__(self):
        self.required_modules = set()
        self.sketchy_modules = set()
        self.builtin_modules = set()
        self.relative_modules = set()
        self.imports = []
        self.import_froms = []
        self.sketchy_nodes = {}
        super(ImportFinder, self).__init__()

    def visit(self, node):
        """Recursively visit all ast nodes.
        Look for Import and ImportFrom nodes. Classify them as being imports
        that are built in, relative, required or questionable. Questionable
        imports are those that occur within the context of a try/except block, a
        function definition or a class definition.
        Parameters
        ----------
        node : ast.AST
            The node to start the recursion
        """
        # add the node to the try/except block to signify that
        # something potentially odd is going on in this import
        if isinstance(node, AST_QUESTIONABLE):
            self.sketchy_nodes[node] = node
        super(ImportFinder, self).visit(node)
        # after the node has been recursed in to, remove the try node
        self.sketchy_nodes.pop(node, None)

    def visit_Import(self, node):
        """Executes when an ast.Import node is encountered
        an ast.Import node is something like 'import bar'
        If ImportCatcher is inside of a try block then the import that has just
        been encountered will be added to the `sketchy_modules` instance
        attribute. Otherwise the module will be added to the `required_modules`
        instance attribute
        """
        self.imports.append(node)
        mods = set([_split(name.name) for name in node.names])
        for mod in mods:
            self._add_import_node(mod)

    def visit_ImportFrom(self, node):
        """Executes when an ast.ImportFrom node is encountered
        an ast.ImportFrom node is something like 'from foo import bar'
        If ImportCatcher is inside of a try block then the import that has just
        been encountered will be added to the `sketchy_modules` instance
        attribute. Otherwise the module will be added to the `required_modules`
        instance attribute
        """
        self.import_froms.append(node)
        if node.module is None:
            # this is a relative import like 'from . import bar'
            # so do nothing
            return
        if node.level > 0:
            # this is a relative import like 'from .foo import bar'
            node_name = _split(node.module)
            self.relative_modules.add(node_name)
            return
        # this is a non-relative import like 'from foo import bar'
        node_name = _split(node.module)
        self._add_import_node(node_name)

    def _add_import_node(self, node_name):
        # see if the module is a builtin
        if node_name in builtin_modules:
            self.builtin_modules.add(node_name)
            return

        # see if we are in a try block
        if self.sketchy_nodes:
            self.sketchy_modules.add(node_name)
            return

        # if none of the above cases are true, it is likely that this
        # ImportFrom node occurs at the top level of the module
        self.required_modules.add(node_name)

    def describe(self):
        """Return the found imports
        Returns
        -------
        dict :
            'required': The modules that were encountered outside of a
                        try/except block
            'questionable': The modules that were encountered inside of a
                            try/except block
            'relative': The modules that were imported via relative import
                        syntax
            'builtin' : The modules that are part of the standard library
        """
        desc =  {
            'required': self.required_modules,
            'relative': self.relative_modules,
            'questionable': self.sketchy_modules,
            'builtin': self.builtin_modules
        }
        desc = {k: v for k, v in desc.items() if v}
        return desc


    def __repr__(self):
        return 'ImportCatcher: %s' % repr(self.describe())

ValueError: __main__.__spec__ is None

These functions are to be used later for any manipulation of the methods that are aforementioned
and listed.

In [89]:
def MethodFinder(imported):
    pass

def print_modules(imported):
    """
    Grab a list of imports from the ImportFinder
    and then prints the detected imported modules.
    """
    print(imported.imports)

def read_py_file(filename=None):
    """
    Read a python file and then parse it into an
    ast representation
    """
    if not filename:
        return

    with open(filename, 'r') as fp:
        data = fp.readlines()
    data = ''.join(data)
    tree = ast.parse(data)
    return tree

Read python file, get imports

In [90]:
a = ImportFinder()
a.imports
def get_modules_used():
    file_content = read_py_file((os.getcwd() +"/testing.py"))
    logging.debug(ast.dump(file_content))
    

In [91]:
get_modules_used()

DEBUG:root:Module(body=[Import(names=[alias(name='numpy', asname='np')]), Import(names=[alias(name='pandas', asname='pd')]), FunctionDef(name='generate_random_array', args=arguments(posonlyargs=[], args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=Call(func=Attribute(value=Attribute(value=Name(id='np', ctx=Load()), attr='random', ctx=Load()), attr='randint', ctx=Load()), args=[Constant(value=0, kind=None), Constant(value=100, kind=None), Tuple(elts=[Constant(value=3, kind=None), Constant(value=6, kind=None), Constant(value=5, kind=None)], ctx=Load())], keywords=[]))], decorator_list=[], returns=None, type_comment=None), Expr(value=Call(func=Name(id='generate_random_array', ctx=Load()), args=[], keywords=[]))], type_ignores=[])


In [92]:
file_content = read_py_file((os.getcwd() +"/testing.py"))
a.visit(file_content)

NameError: name 'pkg_data' is not defined